In [22]:
from keras.models import Model
from keras.applications import VGG16,VGG19
import csv
import csv
import cv2
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.misc import imread
from keras import optimizers
from keras.layers import Flatten, Dense, Lambda, Convolution2D, MaxPooling2D,Input
from sklearn.utils import shuffle
from keras.models import Sequential,load_model
from keras.layers import Activation, Dropout, UpSampling2D, Lambda,Cropping2D,ZeroPadding2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D,Deconvolution2D,Reshape,Add
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras.optimizers import SGD,Adam
import h5py


In [ ]:
#1. Train lane line segmentation model

In [2]:
X_train = pickle.load(open('/floyd/input/highwaysemantic1/X_lanetrain','rb'))
Y_train = pickle.load(open('/floyd/input/highwaysemantic1/Y_lanetrain','rb'))

In [4]:
num_class = 2
img_size = (540, 960, 3)
model = Lenet(img_size,num_class)
    # model = Lenet(img_size,num_class)
    # sgd = SGD(lr=0.0001, decay=0.0005, momentum=0.5)
    # adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    # categorical_crossentropy
model.compile(loss='mse', optimizer='Adam')
model.fit(X_train, Y_train, validation_split=0.2, shuffle=True, nb_epoch=10, batch_size=15)
model.save('./lanebeta.h5')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", name="Conv1", strides=(4, 4), padding="same")`
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((3, 3), strides=(2, 2), name="pool1", padding="same")`
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", name="conv2", strides=(1, 1), padding="same")`
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((3, 3), strides=(2, 2), name="pool2", padding="same")`
  del sys.path[0]
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `C

Train on 251 samples, validate on 63 samples
Epoch 1/10
251/251 [==============================] - 18s 70ms/step - loss: 0.4759 - val_loss: 0.4096
Epoch 2/10
251/251 [==============================] - 10s 41ms/step - loss: 0.2895 - val_loss: 0.1559
Epoch 3/10
251/251 [==============================] - 10s 41ms/step - loss: 0.1045 - val_loss: 0.0633
Epoch 4/10
251/251 [==============================] - 10s 41ms/step - loss: 0.0544 - val_loss: 0.0460
Epoch 5/10
251/251 [==============================] - 10s 41ms/step - loss: 0.0414 - val_loss: 0.0370
Epoch 6/10
251/251 [==============================] - 10s 41ms/step - loss: 0.0370 - val_loss: 0.0343
Epoch 7/10
251/251 [==============================] - 10s 42ms/step - loss: 0.0340 - val_loss: 0.0325
Epoch 8/10
251/251 [==============================] - 10s 42ms/step - loss: 0.0316 - val_loss: 0.0286
Epoch 9/10
251/251 [==============================] - 11s 42ms/step - loss: 0.0299 - val_loss: 0.0272
Epoch 10/10
251/251 [================

In [6]:
model = load_model('./lanebeta.h5')
#sgd = SGD(lr=0.0001, decay=0.0005, momentum=0.5)
#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss = 'mse', optimizer= 'Adam')
model.fit(X_train,Y_train,validation_split = 0.2,shuffle = True,nb_epoch=10,batch_size=8)
model.save('Lenetlanebeta1.h5')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 251 samples, validate on 63 samples
Epoch 1/10
251/251 [==============================] - 16s 63ms/step - loss: 0.0477 - val_loss: 0.0310
Epoch 2/10
251/251 [==============================] - 12s 48ms/step - loss: 0.0286 - val_loss: 0.0254
Epoch 3/10
251/251 [==============================] - 12s 48ms/step - loss: 0.0264 - val_loss: 0.0243
Epoch 4/10
251/251 [==============================] - 12s 48ms/step - loss: 0.0249 - val_loss: 0.0228
Epoch 5/10
251/251 [==============================] - 12s 48ms/step - loss: 0.0236 - val_loss: 0.0219
Epoch 6/10
251/251 [==============================] - 12s 48ms/step - loss: 0.0224 - val_loss: 0.0204
Epoch 7/10
251/251 [==============================] - 12s 48ms/step - loss: 0.0213 - val_loss: 0.0190
Epoch 8/10
251/251 [==============================] - 12s 48ms/step - loss: 0.0203 - val_loss: 0.0183
Epoch 9/10
251/251 [==============================] - 12s 48ms/step - loss: 0.0195 - val_loss: 0.0177
Epoch 10/10
251/251 [================

In [4]:
def Lenet(img_size,num_class):
        #input size:540x960x3; 
    input_img = Input(shape = img_size)
    input_adj = Lambda(lambda x:x/255.0-0.5)(input_img)
    #Encoder:
    #Conv1+pool1
    Conv1 = Convolution2D(24,5,5, border_mode='same',subsample = (4,4), activation='relu',name = 'Conv1')(input_adj)
    Pool1 = MaxPooling2D((3, 3), strides=(2, 2), border_mode='same',name = 'pool1')(Conv1)
    Pool1_skip = Pool1 #skip connection

    #Conv2+pool2
    Conv2 = Convolution2D(48, 5,5, border_mode='same',subsample = (1,1), activation='relu',name = 'conv2')(Pool1)
    Pool2 = MaxPooling2D((3, 3), strides=(2, 2), border_mode='same',name = 'pool2')(Conv2)
    Pool2_skip = Pool2 #skip connection

    #Conv3
    Conv3 = Convolution2D(96,3,3, border_mode='same',subsample = (1,1), activation='relu',name = 'conv3')(Pool2)

    #Conv6 + drop
    Conv4 = Convolution2D(484,11,11, border_mode='same',subsample = (1,1), activation='relu',name = 'conv4')(Conv3)
    Drop4 = Dropout(0.5,name = 'Drop4')(Conv4)

    #Conv7 + drop
    Conv5 = Convolution2D(484,1,1, border_mode='same',subsample = (1,1), activation='relu',name = 'Conv5')(Drop4)
    Drop5 = Dropout(0.5,name = 'Drop5')(Conv5)

    #Conv8 
    Conv6 = Convolution2D(num_class,1,1, border_mode='same',subsample = (1,1), activation='relu',name = 'Conv6')(Drop5)

    #Decoder:

    #skip connection of pool2 and Deconv1:
    Pool2_skip = Lambda(lambda x: x * 1e-1, name='scale_pool1')(Pool2_skip)
    Pool2_skip = Convolution2D(num_class,1,1, border_mode='same',subsample = (1,1), activation='relu',name = 'Pool2_feat')(Pool2_skip)
    Add_1 = Add(name = 'Add1')([Conv6,Pool2_skip])

    #Deconv1:
    Deconv1 = Deconvolution2D(num_class,4,4, subsample=(2, 2), border_mode='same',name = 'Deconv1')(Conv6)
    #Deconv1 = Cropping2D(cropping=((1, 0), (0, 0)),name = 'Deconv1_crop')(Deconv1)
    #skip connection of pool1 and Deconv2:
    Pool1_skip = Lambda(lambda x: x * 1e-2, name='scale_pool1')(Pool1_skip)
    Pool1_skip = Convolution2D(num_class,1,1, border_mode='same',subsample = (1,1), activation='relu',name = 'Pool1_feat')(Pool1_skip)
    Add_2 = Add(name = 'Add2')([Deconv1,Pool1_skip])

    #Deconv3:
    Deconv3 = Deconvolution2D(num_class,16,16, subsample=(8, 8), border_mode='same',name = 'Deconv3')(Add_2)
    Output = Cropping2D(cropping=((4, 0), (0, 0)),name = 'Deconv2_crop')(Deconv3)

    model = Model(input =[input_img],output = [Output])
    return model